In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [86]:
class Mapper:
    def __init__(self, X, y):        
        self._X = X
        self._y = y
    
    def get_maps(self, n_categories, columns=None):
        return self._get_maps(n_categories, columns)[0]
    
    def _get_maps(self, n_categories, columns=None, min_bin_size=3):
        '''
        ----
        n_categories: int or dict
        '''
        class Map:
            def __init__(self, mapping):
                self.mapping = pd.Series(mapping, dtype='Int16', name='codes')
                if pd.isna(self.mapping.index).sum() == 0:
                    self.na_cat_code = int(self.mapping.mean())
                    self.mapping = self.mapping.append(pd.Series({np.nan:self.na_cat_code}))
                else:
                    self.na_cat_code = self.mapping[pd.isna(self.mapping.index)].values[0]
            def __str__(self):
                return 'Map:\nColumn Value -> Category Code\n' + str(self.mapping)
            def __repr__(self):
                return 'Map:\nColumn Value -> Category Code\n' + repr(self.mapping)
            def __call__(self, key):            
                return self[key]
            def __getitem__(self, key):
                try:
                    return self.mapping[key]
                except:
                    return self.na_cat_code

        from pandas.api.types import is_numeric_dtype            
        y = self._y.copy(deep=True)
        df = self._X.copy(deep=True).join(y, how='inner')
        columns = self._X.columns if columns is None else columns
        y_col = y.name    
        mapper, stats = {}, {}
        for col in columns:
            nc = n_categories[col] if type(n_categories) is dict else n_categories
            assert (nc > 1), 'n_categories must be greater than 1'
            if is_numeric_dtype(df[col]) and df[col].nunique() > nc:
                # covert to ordinal
                values = df[col].tolist() + [float('-inf'), float('inf')]            
                cats = pd.qcut(values, nc, duplicates='drop').dtype.categories            
                df[col] = pd.cut(df[col], bins=cats, duplicates='drop')

            # select top categories
            vcounts = df[col].value_counts(dropna=False)
            topn = min(nc, len(vcounts[vcounts >= min_bin_size]))
            keep, setna = set(vcounts.iloc[:topn].index), set(vcounts.iloc[topn:].index)
            mp = {x: x for x in keep}
            mp.update({x: np.nan for x in setna})
            df[col] = df[col].map(mp)

            sum_ser = df.groupby(col, dropna=False)[y_col].agg(sum)
            sum_ser.name = 'sum'
            count_ser = df.groupby(col, dropna=False)[y_col].count()
            count_ser.name = 'count'
            stat_df = pd.merge(count_ser, sum_ser, right_index=True, left_index=True)
            stat_df['ratio'] = stat_df['sum'] / stat_df['count']
            stat_df = stat_df.sort_values(by=['ratio', 'count', col])        
            mapper[col] = Map({v:int(stat_df.iloc[i]['ratio']*100) for i, v in enumerate(stat_df.index)})
            stats[col] = stat_df
        return mapper, stats
    
    def optimal_category_count(self, columns, max_count, min_count=2):
        from math import log
        assert min_count > 1 and max_count > 1
        optimals = {col: min_count for col in columns}
        optimal_scores = {col: None for col in columns}
        for nc in range(min_count, max_count+1):
            _, stats = self._get_maps(nc, columns)
            for col in stats:
                score = 0.0
                stat_df = stats[col]                
                tot = stat_df['count'].sum()
                most_neg = stat_df.iloc[0]['ratio']
                power = max(int(log(len(stat_df), 10)), 2)
                for j in range(1, len(stat_df)):
                    w = stat_df.iloc[j]['count'] / tot
                    v = abs(stat_df.iloc[j]['ratio'] - most_neg)                    
                    score += w * log(1-min(0.9, v))
                most_pos = stat_df.iloc[-1]['ratio']
                for j in range(0, len(stat_df)-1):
                    w = stat_df.iloc[j]['count'] / tot
                    v = abs(stat_df.iloc[j]['ratio'] - most_pos)
                    score += w * log(1-min(0.9, v))
                score = 1 + (score / len(stat_df))
                if optimal_scores[col] is None or score > optimal_scores[col]:
                    optimals[col] = nc
                    optimal_scores[col] = score
        
        return optimals

In [3]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
print(f'train dataset size: {train_df.shape}')
print(f'test dataset size: {test_df.shape}')
print('train columns:')
print(train_df.columns.tolist())
print('test columns:')
print(test_df.columns.tolist())
print('train specific column(s):')
print(set(train_df.columns) - set(test_df.columns))

train dataset size: (891, 12)
test dataset size: (418, 11)
train columns:
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
test columns:
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
train specific column(s):
{'Survived'}


In [4]:
def expand_names(df):    
    df_exp = df.Name.str.split(', ', n=1, expand=True)
    df['Lastname'] = df_exp[0]
    df_exp = df_exp[1].str.split('.', n=1, expand=True)    
    df['Title'] = df_exp[0]    

def expand_ticket(df):    
    s = (df['Ticket'].str.split(expand=True))[0]    
    s[s.str.isdigit()] = np.nan
    df['Ticket_grp'] = s

expand_names(train_df)
expand_names(test_df)
expand_ticket(train_df)
expand_ticket(test_df)

In [5]:
train_df.head(3)

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked   Lastname Title Ticket_grp  
0      0         A/5 21171   7.2500   NaN        S     Braund    Mr        A/5  
1      0          PC 17599  71.2833   C85        C    Cumings   Mrs         PC  
2      0  STON/O2. 3101282   7.9250   NaN        S  Heikkinen  Miss   STON/O2.

In [6]:
test_df.sample(3)

PassengerId  Pclass                                   Name     Sex   Age  \
18           910       3           Ilmakangas, Miss. Ida Livija  female  27.0   
374         1266       1  Dodge, Mrs. Washington (Ruth Vidaver)  female  54.0   
261         1153       3          Nilsson, Mr. August Ferdinand    male  21.0   

     SibSp  Parch            Ticket     Fare Cabin Embarked    Lastname Title  \
18       1      0  STON/O2. 3101270   7.9250   NaN        S  Ilmakangas  Miss   
374      1      1             33638  81.8583   A34        S       Dodge   Mrs   
261      0      0            350410   7.8542   NaN        S     Nilsson    Mr   

    Ticket_grp  
18    STON/O2.  
374        NaN  
261        NaN

In [7]:
class NaiveClassifier:
    def __init__(self):        
        self.model = None
    
    def fit(self, X, y):         
        self.model = None
        max_acc = 0.0
        for col in X:
            pcat, tp1 = list(X.loc[y==1, col].value_counts().items())[0]
            tn1 = X[(X[col] != pcat) & (y==0)].shape[0]
            ncat, tn2 = list(X.loc[y==0, col].value_counts().items())[0]
            tp2 = X[(X[col] != ncat) & (y==1)].shape[0]
            plan = (1, col, pcat) if tp1 + tn1 >= tp2 + tn2 else (0, col, ncat)
            acc = (1/X.shape[0]) * (X[(y==plan[0]) & (X[plan[1]] == plan[2])].shape[0] + 
                                    X[(y==(1-plan[0])) & (X[plan[1]] != plan[2])].shape[0])
            if acc > max_acc:
                self.model = plan
                max_acc = acc
                max_col = col
                
    def predict(self, X):
        assert(self.model != None), 'Classifier is not trained'
        predictions = pd.Series(0, name='predictions', index=X.index)
        label = self.model[0]
        col = self.model[1]
        val = self.model[2]
        predictions[X[col] == val] = label
        predictions[X[col] != val] = 1-label
        return predictions
    
    def __str__(self):
        return f'all X[{self.model[1]}] == {self.model[2]} are {self.model[0]} otherwise {1-self.model[0]}'

In [87]:
%%time

from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

features = ['Sex', 'Age', 'SibSp', 'Parch', 'Ticket_grp', 'Fare', 'Cabin', 'Embarked', 'Title']

y = train_df['Survived'].copy()
X = train_df[features].copy()

models = {'naive_encoded': NaiveClassifier(), 
          'nbayes': GaussianNB(),
          'rf': RandomForestClassifier(), 
          'xgb': XGBClassifier(),
          'svc': make_pipeline(StandardScaler(), SVC()),
          'lr': make_pipeline(StandardScaler(), LogisticRegression()), 
          'knn': make_pipeline(StandardScaler(), KNeighborsClassifier())}

n_splits = 13
result_cols = (['test_baseline', 'train_baseline', 'test_naive', 'train_naive'] + 
               [f'{grp}_{m}' for m in models for grp in ['test', 'train']])
results = pd.DataFrame(columns = result_cols, index = range(n_splits), dtype='float64')
for i in range(n_splits):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=i)    
    results.loc[i, 'train_baseline'] = max(y_train.mean(), 1- y_train.mean())
    results.loc[i, 'test_baseline'] = max(y_valid.mean(), 1- y_valid.mean())
    
    model = NaiveClassifier()
    model.fit(X_train, y_train)
    print(model)
    predictions_train = model.predict(X_train)
    results.loc[i, 'train_naive'] = accuracy_score(predictions_train, y_train)
    predictions_test = model.predict(X_valid)
    results.loc[i, 'test_naive'] = accuracy_score(predictions_test, y_valid)
    
    Xt = X_train.copy()
    Xv = X_valid.copy()    
    mapper = Mapper(X_train, y_train)
    n_categories = mapper.optimal_category_count(features, 128)
    print(n_categories)
    maps = mapper.get_maps(n_categories=n_categories)
    for col in features:
        Xt[col] = X_train[col].map(maps[col])
        Xv[col] = X_valid[col].map(maps[col])

    for name, model in models.items():                        
        model.fit(Xt, y_train)            
        pred_train = model.predict(Xt)
        score_train = accuracy_score(pred_train, y_train)            
        results.loc[i, f'train_{name}'] = score_train
        pred_test = model.predict(Xv)
        score_test = accuracy_score(pred_test, y_valid)
        results.loc[i, f'test_{name}'] = score_test

all X[Title] == Mr are 0 otherwise 1
{'Sex': 2, 'Age': 2, 'SibSp': 2, 'Parch': 2, 'Ticket_grp': 4, 'Fare': 2, 'Cabin': 2, 'Embarked': 2, 'Title': 2}
all X[Title] == Mr are 0 otherwise 1
{'Sex': 2, 'Age': 2, 'SibSp': 2, 'Parch': 2, 'Ticket_grp': 2, 'Fare': 2, 'Cabin': 2, 'Embarked': 2, 'Title': 2}
all X[Sex] == female are 1 otherwise 0
{'Sex': 2, 'Age': 3, 'SibSp': 2, 'Parch': 2, 'Ticket_grp': 3, 'Fare': 2, 'Cabin': 4, 'Embarked': 2, 'Title': 2}
all X[Sex] == female are 1 otherwise 0
{'Sex': 2, 'Age': 2, 'SibSp': 2, 'Parch': 2, 'Ticket_grp': 4, 'Fare': 2, 'Cabin': 4, 'Embarked': 2, 'Title': 2}
all X[Sex] == female are 1 otherwise 0
{'Sex': 2, 'Age': 2, 'SibSp': 2, 'Parch': 2, 'Ticket_grp': 3, 'Fare': 2, 'Cabin': 2, 'Embarked': 2, 'Title': 2}
all X[Sex] == female are 1 otherwise 0
{'Sex': 2, 'Age': 2, 'SibSp': 2, 'Parch': 2, 'Ticket_grp': 3, 'Fare': 2, 'Cabin': 2, 'Embarked': 2, 'Title': 2}
all X[Sex] == female are 1 otherwise 0
{'Sex': 2, 'Age': 2, 'SibSp': 2, 'Parch': 2, 'Ticket_grp': 

In [88]:
# 2
resfsafd = results
cols_train = results.columns[results.columns.str.contains('train')]
cols_test = results.columns[results.columns.str.contains('test')]
results[cols_test].apply(lambda x: x- results['test_naive']).describe().T.sort_values(by='mean', ascending=False)

count      mean       std       min       25%       50%  \
test_xgb             13.0  0.005864  0.027362 -0.040359 -0.017937  0.008969   
test_lr              13.0  0.005519  0.013596 -0.017937 -0.004484  0.004484   
test_rf              13.0  0.004829  0.025725 -0.049327 -0.004484  0.013453   
test_svc             13.0  0.004829  0.012344 -0.022422  0.000000  0.004484   
test_naive           13.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_naive_encoded   13.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_knn             13.0 -0.009314  0.028388 -0.071749 -0.017937 -0.004484   
test_nbayes          13.0 -0.024491  0.056622 -0.147982 -0.040359  0.000000   
test_baseline        13.0 -0.164884  0.023598 -0.192825 -0.183857 -0.170404   

                         75%       max  
test_xgb            0.026906  0.040359  
test_lr             0.008969  0.031390  
test_rf             0.017937  0.040359  
test_svc            0.008969  0.026906  
test_naive          0.000000  0.000000  
test_naive_encoded  0.000000  0.000000  
test_knn            0.013453  0.026906  
test_nbayes         0.004484  0.022422  
test_baseline      -0.143498 -0.125561

In [66]:
# reslognosum
resreslognosum = results
cols_train = results.columns[results.columns.str.contains('train')]
cols_test = results.columns[results.columns.str.contains('test')]
results[cols_test].apply(lambda x: x- results['test_naive']).describe().T.sort_values(by='mean', ascending=False)

count      mean       std       min       25%       50%  \
test_svc             13.0  0.032425  0.013472  0.004484  0.026906  0.035874   
test_xgb             13.0  0.011038  0.030338 -0.035874 -0.004484  0.013453   
test_knn             13.0  0.009659  0.023290 -0.022422 -0.004484  0.008969   
test_rf              13.0  0.007589  0.016717 -0.022422 -0.004484  0.008969   
test_lr              13.0  0.007244  0.019593 -0.022422 -0.008969  0.000000   
test_naive           13.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_naive_encoded   13.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_nbayes          13.0 -0.003105  0.028678 -0.067265 -0.022422 -0.008969   
test_baseline        13.0 -0.164884  0.023598 -0.192825 -0.183857 -0.170404   

                         75%       max  
test_svc            0.040359  0.053812  
test_xgb            0.026906  0.067265  
test_knn            0.026906  0.053812  
test_rf             0.017937  0.031390  
test_lr             0.022422  0.035874  
test_naive          0.000000  0.000000  
test_naive_encoded  0.000000  0.000000  
test_nbayes         0.022422  0.035874  
test_baseline      -0.143498 -0.125561

In [63]:
# log
reslog
cols_train = results.columns[results.columns.str.contains('train')]
cols_test = results.columns[results.columns.str.contains('test')]
results[cols_test].apply(lambda x: x- results['test_naive']).describe().T.sort_values(by='mean', ascending=False)

count      mean       std       min       25%       50%  \
test_svc             13.0  0.027251  0.012744  0.008969  0.017937  0.022422   
test_lr              13.0  0.016902  0.009885  0.004484  0.008969  0.017937   
test_xgb             13.0  0.011728  0.021933 -0.035874  0.000000  0.017937   
test_knn             13.0  0.007244  0.028096 -0.026906 -0.022422  0.008969   
test_rf              13.0  0.004139  0.026622 -0.040359 -0.008969  0.008969   
test_naive           13.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_naive_encoded   13.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_nbayes          13.0 -0.064505  0.084064 -0.170404 -0.147982 -0.031390   
test_baseline        13.0 -0.164884  0.023598 -0.192825 -0.183857 -0.170404   

                         75%       max  
test_svc            0.035874  0.049327  
test_lr             0.022422  0.031390  
test_xgb            0.022422  0.053812  
test_knn            0.026906  0.058296  
test_rf             0.013453  0.058296  
test_naive          0.000000  0.000000  
test_naive_encoded  0.000000  0.000000  
test_nbayes         0.004484  0.053812  
test_baseline      -0.143498 -0.125561

In [53]:
# 10
res10
cols_train = results.columns[results.columns.str.contains('train')]
cols_test = results.columns[results.columns.str.contains('test')]
results[cols_test].apply(lambda x: x- results['test_naive']).describe().T.sort_values(by='mean', ascending=False)

count      mean       std       min       25%       50%  \
test_svc             13.0  0.039324  0.010541  0.022422  0.031390  0.040359   
test_rf              13.0  0.016212  0.013015 -0.004484  0.004484  0.017937   
test_lr              13.0  0.014143  0.012663 -0.004484  0.004484  0.013453   
test_xgb             13.0  0.013798  0.024388 -0.026906  0.000000  0.013453   
test_knn             13.0  0.012418  0.024159 -0.031390  0.000000  0.008969   
test_naive           13.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_naive_encoded   13.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_nbayes          13.0 -0.001035  0.027834 -0.058296 -0.013453  0.000000   
test_baseline        13.0 -0.164884  0.023598 -0.192825 -0.183857 -0.170404   

                         75%       max  
test_svc            0.044843  0.062780  
test_rf             0.022422  0.040359  
test_lr             0.022422  0.031390  
test_xgb            0.022422  0.053812  
test_knn            0.026906  0.053812  
test_naive          0.000000  0.000000  
test_naive_encoded  0.000000  0.000000  
test_nbayes         0.013453  0.040359  
test_baseline      -0.143498 -0.125561

In [26]:
cols_train = results.columns[results.columns.str.contains('train')]
cols_test = results.columns[results.columns.str.contains('test')]
topcols = sorted(set(results[cols_test].T.idxmax().values))
results[topcols].apply(lambda x: x - results['naive_test']).describe()

rf_128_test  rf_32_test  rf_8_test  svc_128_test  svc_8_test  \
count    10.000000   10.000000  10.000000  1.000000e+01   10.000000   
mean      0.025140    0.029609   0.021788  4.469274e-03    0.034637   
std       0.025702    0.028614   0.033875  1.931665e-02    0.026177   
min      -0.033520   -0.022346  -0.027933 -2.234637e-02   -0.005587   
25%       0.018156    0.013966  -0.006983 -1.117318e-02    0.019553   
50%       0.030726    0.036313   0.027933  5.551115e-17    0.039106   
75%       0.037709    0.047486   0.036313  2.234637e-02    0.054469   
max       0.061453    0.067039   0.083799  3.351955e-02    0.067039   

       xgb_32_test  xgb_8_test  
count    10.000000   10.000000  
mean      0.024022    0.022905  
std       0.026863    0.025458  
min      -0.011173   -0.022346  
25%       0.011173    0.001397  
50%       0.022346    0.030726  
75%       0.039106    0.043296  
max       0.078212    0.055866

In [ ]:
# Using cross validation

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, ShuffleSplit, GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

features = ['Sex', 'Age', 'SibSp', 'Parch', 'Ticket_grp', 'Fare', 'Cabin', 'Embarked', 'Lastname', 'Title']

y_train = train_df['Survived'].copy()
X_train = train_df[features].copy()
X_test = test_df[features].copy()

my_mapping, _ = get_mapper(X_train, y_train, n_categories=n_categories, columns=features)

for col in features:
    X_train[col] = X_train[col].map(my_mapping[col])
    X_test[col] = X_test[col].map(my_mapping[col])

# my_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), 
#                             RandomForestClassifier(criterion='gini', n_estimators=300, random_state=0, max_depth=8))
my_pipeline = RandomForestClassifier(criterion='gini', n_estimators=300, random_state=0, max_depth=8)
cv = ShuffleSplit(n_splits=40, test_size=.46)
scores = cross_val_score(my_pipeline, X_train, y_train, cv=cv, scoring='accuracy')
print(f'Model accuracy: {scores.mean()}')

In [ ]:
my_pipeline.fit(X_train, y_train)
predictions = my_pipeline.predict(X_test)
output_df = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': predictions})
# output_df.to_csv('my_submission_4.csv', index=False)